In [12]:
from FakeModel import Model

from textblob import TextBlob
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper

In [13]:
editor = Editor()

In [16]:
TextBlob("Going out to eat is not a bad option").sentiment[0]

0.3499999999999999

## Example formatting for FakeModel

In [17]:
model_instance = Model("abc")
res = model_instance()

print(res)

{'PE': 'Acceptable', 'KE': 'Insufficient', 'LCE': 'Acceptable'}


# Vocab + POS

### Minimum Functionality Test
#### Check for Correct definitions